In [ ]:
# отключим предупреждения Anaconda
import warnings
warnings.simplefilter('ignore')

# будем отображать графики прямо в jupyter'e
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt
#графики в svg выглядят более четкими
%config InlineBackend.figure_format = 'svg' 

#увеличим дефолтный размер графиков
from pylab import rcParams
rcParams['figure.figsize'] = 8, 5
import pandas as pd

In [ ]:
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly
import plotly.graph_objs as go

init_notebook_mode(connected=True)

In [ ]:
#10 vertical divs
#25 subdivs
#2500 points verical
#2**16 values
#0.75 V/div
#7.5 p/p
#-3.75 - 3.75 volts
#-32768 - 32767 data
#(x - in_min) * (out_max - out_min) / (in_max - in_min) + out_min;
#add 0.75 volts since zero level below zero
def data_to_volts(x):
    return(x + 32768) * 7.5 / 2**16 - 3

In [ ]:
from datetime import datetime as dt
data=[]
time=[]
folder="12#04#2019#14#48#19"
for i in range(10,81,10):
    file_data=[]
    file_times=[]
    with open("{}/test_{}_{}.txt".format(folder,i,folder), "r") as f:        
        file_data=eval(f.readline())
        file_times=eval(f.readline())
    data.append(file_data)
    time.append([dt.strptime(s,"%d#%m#%Y#%H#%M#%S#%f") for s in file_times])

In [ ]:
time_delta=[[(i-t[0]).total_seconds() for i in t]for t in time]

In [ ]:
import numpy as np
import pandas as pd

df=[]#dataframes for basic pictures
df2=[]#dataframes for screenshot to screenshot pictures
for i in data:
    npdata=np.array(i)
    df.append(pd.DataFrame(npdata.transpose()))
    #df2.append(pd.DataFrame(npdata))

In [ ]:
for dfidx in range(len(df)):
    datas =[go.Scatter(
            x=df[dfidx].index,
            y=df[dfidx][i],
            name='{}'.format(time_delta[dfidx][i])
            ) for i in range(1,len(time_delta[dfidx]),50)]
    layout = go.Layout(
        title='Raw oscilloscope snapshots - data{}'.format(dfidx),
        xaxis=dict(
            title="TICKS",
            showgrid=True,
            zeroline=True,
            showline=True,
            mirror='ticks',
            gridcolor='#bdbdbd',
            gridwidth=2,
            zerolinecolor='#969696',
            zerolinewidth=4,
            linecolor='#636363',
            linewidth=6
        ),
        yaxis=dict(
            title="POINTS",
            showgrid=True,
            zeroline=True,
            showline=True,
            mirror='ticks',
            gridcolor='#bdbdbd',
            gridwidth=2,
            zerolinecolor='#969696',
            zerolinewidth=4,
            linecolor='#636363',
            linewidth=6
        )
    )

    # cоздаем объект Figure и визуализируем его
    fig = go.Figure(data=datas, layout=layout)
    iplot(fig, show_link=False)


In [ ]:
#convert ticks to times
#convert points to voltage
for i in range(len(df)):
    df[i].index=df[i].index*20*10**-6
    df[i]=df[i].applymap(data_to_volts)
    df2.append(df[i])

In [ ]:
for dfidx in range(len(df)):
    datas =[go.Scatter(
            x=df[dfidx].index,
            y=df[dfidx][i],
            name='{}'.format(time_delta[dfidx][i])
            ) for i in range(1,len(time_delta[dfidx]),50)]
    layout = go.Layout(
        title='Raw oscilloscope snapshots - data{}'.format(dfidx),
        xaxis=dict(
            title="TIME",
            showgrid=True,
            zeroline=True,
            showline=True,
            mirror='ticks',
            gridcolor='#bdbdbd',
            gridwidth=2,
            zerolinecolor='#969696',
            zerolinewidth=4,
            linecolor='#636363',
            linewidth=6
        ),
        yaxis=dict(
            title="VOLTS",
            showgrid=True,
            zeroline=True,
            showline=True,
            mirror='ticks',
            gridcolor='#bdbdbd',
            gridwidth=2,
            zerolinecolor='#969696',
            zerolinewidth=4,
            linecolor='#636363',
            linewidth=6
        )
    )

    # cоздаем объект Figure и визуализируем его
    fig = go.Figure(data=datas, layout=layout)
    iplot(fig, show_link=False)

In [ ]:
df_filt=[]
df2_filt=[]
for d in df:
    temp=d.sub(d[1], axis='rows')
    temp=d.rolling(200,win_type="gaussian").mean(std=3)
    df_filt.append(temp)
    df2_filt.append(temp.transpose()[1:])

In [ ]:
for dfidx in range(len(df)):
    datas =[go.Scatter(
            x=df_filt[dfidx].index,
            y=df_filt[dfidx][i],
            name='{}'.format(time_delta[dfidx][i])
            ) for i in range(1,len(time_delta[dfidx]),50)]
    layout = go.Layout(
        title='Filtered oscilloscope snapshots - data{}'.format(dfidx),
        xaxis=dict(
            title="TIME",
            showgrid=True,
            zeroline=True,
            showline=True,
            mirror='ticks',
            gridcolor='#bdbdbd',
            gridwidth=2,
            zerolinecolor='#969696',
            zerolinewidth=4,
            linecolor='#636363',
            linewidth=6
        ),
        yaxis=dict(
            title="VOLTAGE",
            showgrid=True,
            zeroline=True,
            showline=True,
            mirror='ticks',
            gridcolor='#bdbdbd',
            gridwidth=2,
            zerolinecolor='#969696',
            zerolinewidth=4,
            linecolor='#636363',
            linewidth=6
        )
    )

    # cоздаем объект Figure и визуализируем его
    fig = go.Figure(data=datas, layout=layout)
    iplot(fig, show_link=False)

In [ ]:
point_list=[0.0072,0.0172,0.0272,0.0372,0.0472]

In [ ]:
for dfidx in point_list:
    datas =[go.Scatter(
            x=time_delta[i],
            y=df2_filt[i][dfidx],
            name='{}'.format(i)
            ) for i in range(len(df2_filt))]
    layout = go.Layout(
        title='Volts change at sigle screenshot time - data{}'.format(dfidx),
        xaxis=dict(
            title="TIME",
            showgrid=True,
            zeroline=True,
            showline=True,
            mirror='ticks',
            gridcolor='#bdbdbd',
            gridwidth=2,
            zerolinecolor='#969696',
            zerolinewidth=4,
            linecolor='#636363',
            linewidth=6
        ),
        yaxis=dict(
            title="VOLTAGE",
            showgrid=True,
            zeroline=True,
            showline=True,
            mirror='ticks',
            gridcolor='#bdbdbd',
            gridwidth=2,
            zerolinecolor='#969696',
            zerolinewidth=4,
            linecolor='#636363',
            linewidth=6
        )
    )

    # cоздаем объект Figure и визуализируем его
    fig = go.Figure(data=datas, layout=layout)
    iplot(fig, show_link=False)

In [ ]:
def volts_to_res(u_signal,u_res):
    return u_signal*1.2*10**6/u_res

In [ ]:
df2_resistance = []
for d in df2_filt:
    df2_resistance.append(d.applymap(lambda x:volts_to_res(0.8,x)))

In [ ]:
for dfidx in point_list:
    datas =[go.Scatter(
            x=time_delta[i],
            y=df2_resistance[i][dfidx],
            name='{}'.format(i)
            ) for i in range(len(df2_filt))]
    layout = go.Layout(
        title='Resistance change at sigle screenshot time - data{}'.format(dfidx),
        xaxis=dict(
            title="TIME",
            showgrid=True,
            zeroline=True,
            showline=True,
            mirror='ticks',
            gridcolor='#bdbdbd',
            gridwidth=2,
            zerolinecolor='#969696',
            zerolinewidth=4,
            linecolor='#636363',
            linewidth=6
        ),
        yaxis=dict(
            title="OHMS",
            showgrid=True,
            zeroline=True,
            showline=True,
            mirror='ticks',
            gridcolor='#bdbdbd',
            gridwidth=2,
            zerolinecolor='#969696',
            zerolinewidth=4,
            linecolor='#636363',
            linewidth=6
        )
    )

    # cоздаем объект Figure и визуализируем его
    fig = go.Figure(data=datas, layout=layout)
    iplot(fig, show_link=False)